
##Introduction 

It is simple approach to rate severity of toxic comments. And find probability of the comment to be toxic one and give the score based on the probability.
The final score is the probability predicted by the Model.

Notebook is build upon this public kernel: https://www.kaggle.com/datasets/devkhant24/toxic-comment

In [ ]:
# Importing libraries

import math
import os
import random
import numpy as np
import pandas as pd
import re
import unidecode

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from imblearn.under_sampling import RandomUnderSampler

import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, GRU, Embedding, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

 Defining constants

In [ ]:

voc_size = 50000
max_sequence_length = 250
embedding_dim = 100
Batch_size = 16

train_prev_comp = "../input/toxic-comment/jigsaw-toxic-comment-train.csv"
test_cur_comp = "../input/jigsaw-toxic-severity-rating/comments_to_score.csv"


def seed_everything():
    np.random.seed(123)
    random.seed(123)
    tf.random.set_seed(123)
    os.environ["TF_CPP_MIN_LOG_LEVEL"] = '2'
    os.environ['PYTHONHASHSEED'] = str(123)

seed_everything()

In [ ]:
# Function for cleaning comments

def clean_data(data):
    final = []
    for sent in data:
        sent = sent.replace('\\n', ' ').replace('\n', ' ').replace('\t',' ').replace('\\', ' ').replace('. com', '.com')
        soup = BeautifulSoup(sent, "html.parser")
        sent = soup.get_text(separator=" ")
        remove_https = re.sub(r'http\S+', '', sent)
        sent = re.sub(r"\ [A-Za-z]*\.com", " ", remove_https)
        sent = unidecode.unidecode(sent)
        sent = sent.lower()
        sent = re.sub(r"[^a-zA-Z0-9:$-,()%.?!]+", ' ', sent) 
        sent = re.sub(r"[:$-,()%.?!]+", ' ',sent)
        stoplist = stopwords.words("english")
        sent = [word for word in word_tokenize(sent) if word not in stoplist]
        sent = " ".join(sent)
        final.append(sent)
    
    return final

In [ ]:
# Reading train file from previous competition

df = pd.read_csv(train_prev_comp)


df["y"] = (df[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].sum(axis=1) > 0).astype(int)
df.drop(["id","toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"], axis=1, inplace = True)
df.head()

,comment_text,y
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [ ]:
# Seeing that dataset is imbalanced

df["y"].value_counts()

0    201081
1     22468
Name: y, dtype: int64

Balacing dataset

In [ ]:

X = np.array(df["comment_text"].values)
X = X.reshape(-1,1)
y = np.array(df["y"].values)
rus = RandomUnderSampler(random_state=0)
x, y = rus.fit_resample(X, y)

x = x.flatten()
df = pd.DataFrame()
df["text"] = x
df["target"] = y


# Now its balanced

df["target"].value_counts()

0    22468
1    22468
Name: target, dtype: int64

In [ ]:
# Creating column clean_text for cleaned comments

df["text"] = clean_data(df["text"])

Defining keras Model with GRU units

In [ ]:

class GRU_model(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.Embedding = Embedding(voc_size, embedding_dim, input_length = max_sequence_length)
        self.GRU1 = Bidirectional(GRU(128, return_sequences=True))
        self.Dropout1 = Dropout(0.25)
        self.GRU2 = Bidirectional(GRU(64, return_sequences = False))
        self.Dropout2 = Dropout(0.25)
        self.Dense1 = Dense(64, activation="relu")
        self.Dropout3 = Dropout(0.2)
        self.Dense2 = Dense(1, activation="sigmoid")
    
    def call(self, inputs):
        x = self.Embedding(inputs)
        x = self.GRU1(x)
        x = self.Dropout1(x)
        x = self.GRU2(x)
        x = self.Dropout2(x)
        x = self.Dense1(x)
        x = self.Dropout3(x)
        x = self.Dense2(x)
        
        return x

In [ ]:
# Using early_stopping as callback function 
# It takes the weigths of epoch with the best val_accuracy

early_stopping = EarlyStopping(patience = 5,restore_best_weights = True)

Tokenizing the comments from train dataset

In [ ]:

tokenizer = Tokenizer(num_words = voc_size)
tokenizer.fit_on_texts(df["text"].values)
X = tokenizer.texts_to_sequences(df["text"].values)
X = pad_sequences(X, maxlen = max_sequence_length)

In [ ]:
model = GRU_model()
model.compile(
        loss = tf.keras.losses.BinaryCrossentropy(),
        optimizer = "Adam",
        metrics = ["accuracy"]
    )

model.fit(
        X, 
        df.target, 
        epochs = 10, 
        validation_split = 0.2,
        batch_size = Batch_size, 
        callbacks = [early_stopping]
    )

Epoch 1/10
2247/2247 [==============================] - 101s 42ms/step - loss: 0.2942 - accuracy: 0.8761 - val_loss: 0.2347 - val_accuracy: 0.9227
Epoch 2/10
2247/2247 [==============================] - 94s 42ms/step - loss: 0.1639 - accuracy: 0.9375 - val_loss: 0.2342 - val_accuracy: 0.9211
Epoch 3/10
2247/2247 [==============================] - 94s 42ms/step - loss: 0.0894 - accuracy: 0.9676 - val_loss: 0.2741 - val_accuracy: 0.9000
Epoch 4/10
2247/2247 [==============================] - 94s 42ms/step - loss: 0.0501 - accuracy: 0.9830 - val_loss: 0.4452 - val_accuracy: 0.8856
Epoch 5/10
2247/2247 [==============================] - 94s 42ms/step - loss: 0.0312 - accuracy: 0.9898 - val_loss: 0.6138 - val_accuracy: 0.8623
Epoch 6/10
2247/2247 [==============================] - 94s 42ms/step - loss: 0.0215 - accuracy: 0.9927 - val_loss: 0.5662 - val_accuracy: 0.8569
Epoch 7/10
2247/2247 [==============================] - 94s 42ms/step - loss: 0.0147 - accuracy: 0.9951 - val_loss: 0.4859 

In [ ]:
# Reading given test dataset 

test = pd.read_csv(test_cur_comp)

test["text"] = clean_data(test["text"])
x_test = tokenizer.texts_to_sequences(test["text"].values)
x_test = pad_sequences(x_test, maxlen = max_sequence_length)

pred = model.predict(x_test)

In [ ]:
# Making submission file

final = pd.DataFrame()
final["comment_id"] = test["comment_id"]
final["score"] = pred
final.to_csv("submission.csv", index=False)

In [ ]:
final.head()

,comment_id,score
0,114890,0.001387
1,732895,0.007995
2,1139051,0.001851
3,1434512,0.014324
4,2084821,0.828060
